In [1]:
import itertools
import warnings

# Our numerical workhorses
import numpy as np
import pandas as pd
import scipy.stats as st
import scipy.special

# The MCMC Hammer
import emcee

# BE/Bi 103 utilities
import bebi103

# Import plotting tools
import matplotlib.pyplot as plt
import seaborn as sns
import corner

# Magic function to make matplotlib inline; other style specs must come AFTER
%matplotlib inline

# This enables high res graphics inline (only use with static plots (non-Bokeh))
# SVG is preferred, but there is a bug in Jupyter with vertical lines
%config InlineBackend.figure_formats = {'png', 'retina'}

# JB's favorite Seaborn settings for notebooks
rc = {'lines.linewidth': 2, 
      'axes.labelsize': 18, 
      'axes.titlesize': 18, 
      'axes.facecolor': 'DFDFE5'}
sns.set_context('notebook', rc=rc)
sns.set_style('darkgrid', rc=rc)

# Suppress future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
df = pd.read_csv("./data/gardner_hw6/gardner_mt_catastrophe_only_tubulin.csv", comment = "#")

In [3]:
df.head()

,12 uM,7 uM,9 uM,10 uM,14 uM
0,25.000,35,25,50,60
1,40.000,45,40,60,75
2,40.000,50,40,60,75
3,45.429,50,45,75,85
4,50.000,55,50,75,115


In [4]:
def log_likelihood1(tau, t, m):
    """
    Takes in tau (should be an array), 
    data, various values for m. We are very proud of this function.
    """
    tau = np.sort(tau)
    
    num_handler = [] #numerator
    den_handler = [] #denominator
    den_elements = [] #elements necessary to take products in den
    func_handler = [] #output log posterior
    
    for j in range(m):
        num_handler.append(tau[j]**(m-2) * np.exp(-t/tau[j]))

        for k in range(m):
            if k != j:
                den_elements.append(tau[j] - tau[k])

        den_handler.append(np.prod(den_elements))

        func_handler.append(num_handler[j]/den_handler[j])
        den_elements = []
    
    #print(func_handler)

    if np.sum(func_handler)>0:
        return np.log(np.sum(func_handler))

    else:
        #raise RuntimeError("WTF Man.")
        return -np.inf

def log_prior1(tau, m):
    """
    Log prior for model defined above. Takes in m and tau, 
    returns log prior.
    """
    tau = np.sort(tau)
    
    for i in range(m-1):

        if not tau[i]<tau[i+1]:

            return -np.inf
    
    if not np.all(tau> 0):
        return -np.inf
    
    return -np.log(np.prod(tau))
        
#test_run = log_likelihood(tau, t, 3)

def tau_start1(m, n_walkers, n_temps):
    p = np.empty((n_temps, n_walkers, m))
    
    for i in range(m):
        p[:,:,i]=np.random.exponential(1, (n_temps,n_walkers))
    return p



In [5]:
def log_likelihood(tau, t, m):
    """
    Takes in tau (should be an array), 
    data, various values for m. We are very proud of this function.
    """
    tau = np.sort(tau)
    logpost = np.empty(len(t))
    for i in range(len(t)):
        
        num_handler = np.empty( m) #numerator
        exp_handler = np.empty(m)
        den_handler = np.empty(m) #denominator
        den_elements = [] #elements necessary to take products in den
        func_handler = np.empty(m) #output log posterior

        for j in range(m):
            num_handler[j] = tau[j]**(m-2)
            exp_handler[j] = -t[i]/tau[j]
            for k in range(m):
                if k != j:
                    den_elements.append(tau[j] - tau[k])

            den_handler[j] = np.prod(den_elements)
            den_elements = []

        scaling_factor = num_handler / den_handler
        func_handler = scaling_factor * np.exp(exp_handler)   
        
        #if np.sum(func_handler)>0:
        logpost[i] = scipy.misc.logsumexp(exp_handler, b=scaling_factor)            
            
    return np.sum(logpost)

def log_prior(tau, m):
    """
    Log prior for model defined above. Takes in m and tau, 
    returns log prior.
    """
    if not np.all(tau > 0):
        return -np.inf
    
    return -np.log(np.prod(tau))
        
def tau_start(m, n_walkers, n_temps):
    p = np.empty((n_temps, n_walkers, m))
    
    for i in range(m):
        p[:,:,i] = np.random.exponential(300, (n_temps,n_walkers))
    return p

In [6]:
tau = [15, 10, 17, 1, 4,6 ,20,8,9,11]
t = df['12 uM']
m = 10

test = log_likelihood(tau, t, m)

In [7]:
test

-11223.694211888453

In [5]:
n_burn = 100
n_steps = 500
n_temps = 10
n_walkers = 10
m = 3
p0=tau_start(m, n_walkers, n_temps)
loglargs = (df["12 uM"], m)
logpargs = (m,)
columns = ["tau_1", "tau_2", "tau_3"]

In [ ]:
import time
start = time.time()

df_out, lnZ_out, dlnZ_out = bebi103.run_pt_emcee(log_likelihood, log_prior, n_burn, n_steps, 
                    n_temps=n_temps, p0=p0, loglargs=loglargs, 
                    logpargs=logpargs, threads=None, columns=columns,
                    return_lnZ=True)

print('It took ', time.time()-start, ' seconds.')

In [ ]:
lnZ_out

In [ ]:
df_out.to_csv('./ptmcmc_results_test1.csv')

In [ ]:
a, b, c=5, 3, 2

In [ ]:
np.log(np.exp(a) - np.exp(b) + np.exp(c))

In [ ]:
a + np.log(1 - np.exp(b-a) + np.exp(c-a))

In [ ]:
t = df["12 uM"][0]

In [ ]:
df_out

In [ ]:
tau = np.array([0.06463911, 0.04257534, 0.11927362])
z=log_likelihood(tau,t,3)

In [ ]:
z